# Analysis of District-wide Standardized Test Results
This report analyzes 40,000 9th-12th grade student's math and reading scores, along with key information about the 15 District schools that they attend. 
<br>This data has been aggregated to highlight key trends in school performance, and I have noted some of the most interesting trends that I observed here:
- District students perform **significantly better in Reading**, on average, than they do in Math
    - Notably, *this is not true at the top performing schools* - these students achieve balanced scores across Math and Reading
- The difference in student performance is stark between the top-performing and the worst-performing schools:<br>
    - **90%** of students at the former pass *both* Math and Reading, whereas **HALF** of the students at the lowest performing schools *pass neither*.
- Surprisingly, contrary to popular opinion:
    - Schools with a higher average spending budget per student **actually perform worse** than schools with lower average spending. In fact, it would appear, that the more money spent the worse the scores!
- And as one might expect:
    - Charter schools perform *significantly better* than District schools in these academic metrics.
    - The large schools perform *significantly worse* than the small & medium-sized schools.


The tables and detailed analysis of this data can be viewed below:
1. [The District Summary Report](#District-Summary) 
2. [The School Summary Report](#School-Summary)
3. [The Top Performing Schools](#Top-Performing-Schools-(By-%-Overall-Passing))
4. [The Bottom Performing Schools](#Bottom-Performing-Schools-(By-%-Overall-Passing))
5. [Math Scores by Grade](#Math-Scores-by-Grade)
6. [Reading Scores by Grade](#Reading-Scores-by-Grade)
7. [Scores by School Spending](#Scores-by-School-Spending)
8. [Scores by School Size](#Scores-by-School-Size)
9. [Scores by School Type](#Scores-by-School-Type)






In [1]:
# Dependencies and Setup
import pandas as pd

# Files to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
schools_df = pd.merge(student_data, school_data, how="left", on=["school_name"])


## District Summary


In [2]:
# PREPARE VALUES FOR THE DISTRICT SUMMARY REPORT
# Calculate the total number of schools
num_schools = len(schools_df['School ID'].unique())

# Calculate total number of students
num_students = len(schools_df['Student ID'].unique())

# Calculate the total budget
tot_budget = school_data['budget'].sum()

# Calculate the average math score
avg_math_score = schools_df['math_score'].mean()

# Calculate the average reading score
avg_reading_score = schools_df['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater) 
pct_passing_math = (len(schools_df.loc[schools_df['math_score'] >= 70])/num_students)*100

# Calculate the percentage of students with a passing reading score (70 or greater)
pct_passing_reading = (len(schools_df.loc[schools_df['reading_score'] >= 70])/num_students)*100

# Calculate the percentage of students who passed math and reading (% Overall Passing)
pct_overall_passing = (len(schools_df.loc[(schools_df['reading_score'] >= 70) & (schools_df['math_score'] >= 70)])/num_students)*100


In [3]:
# Creating a summary DataFrame for the District
district_summary_df = pd.DataFrame({'Total Schools': [num_schools], 
                                     'Total Students': [num_students],
                                     'Total Budget': [tot_budget],
                                     'Average Math Score': [avg_math_score],
                                     'Average Reading Score': [avg_reading_score],
                                     '% Passing Math': [pct_passing_math],
                                     '% Passing Reading': [pct_passing_reading],
                                     '% Overall Passing': [pct_overall_passing],
                                    })


In [4]:
# format the table for better readability
district_summary_df['Total Students'] = district_summary_df['Total Students'].map("{:,}".format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,.2f}".format)
district_summary_df.head()
district_summary_df.style.hide_index()


Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [5]:
# PREPARE VALUES FOR THE SCHOOL SUMMARY REPORT
school_group = schools_df.groupby(["school_name"])

# Total students
student_counts = school_group["Student ID"].count()

# total school budget
total_school_budget = school_group['budget'].mean()

# per student budget
per_student_budget = school_group['budget'].mean()/school_group["Student ID"].count() 

# average math score
average_math_score = school_group["math_score"].mean()

# average reading score
average_reading_score = school_group["reading_score"].mean()

# percent passing math
number_passing_math = schools_df.groupby('school_name')['math_score'].apply(lambda x: x[x >= 70].count())
pct_passing_math = (number_passing_math/student_counts)*100

# percent passing reading
number_passing_reading = schools_df.groupby('school_name')['reading_score'].apply(lambda x: x[x >= 70].count())
pct_passing_reading = (number_passing_reading/student_counts)*100

# percent passing both
passing_both_df = schools_df.loc[(schools_df['math_score'] >= 70) & (schools_df['reading_score'] >= 70)]
number_passing_both = passing_both_df.groupby('school_name')['Student ID'].count()
pct_passing_both = (number_passing_both/student_counts)*100



In [6]:
# Creating the School summary DataFrame
school_summary_df = pd.DataFrame({   'Total Students': student_counts,
                                     'Total School Budget': total_school_budget,
                                     'Per Student Budget': per_student_budget,
                                     'Average Math Score': average_math_score,
                                     'Average Reading Score': average_reading_score,
                                     '% Passing Math': pct_passing_math,
                                     '% Passing Reading': pct_passing_reading,
                                     '% Overall Passing': pct_passing_both
                                    })

# Clean up the School summary dataframe
merged_school_summary_df = pd.merge(school_summary_df,school_data[['school_name','type']],on='school_name', how='left')
merged_school_summary_df.head()
reorganized_school_summary_df = merged_school_summary_df[['school_name', 'type','Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing']]
final_school_summary_df = reorganized_school_summary_df.rename(columns={'school_name':'School Name', 'type':'School Type'})


In [7]:
# format the table for better readability
final_school_summary_df['Total School Budget'] = final_school_summary_df['Total School Budget'].map("${:,.2f}".format)
final_school_summary_df['Per Student Budget'] = final_school_summary_df['Per Student Budget'].map("${:,.2f}".format)
final_school_summary_df.head(20)
final_school_summary_df.style.hide_index()


School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [8]:
# Sort and display the top five performing schools by % overall passing
top_schools_df = final_school_summary_df.sort_values("% Overall Passing", ascending=False, ignore_index=True)
top_schools_df.head(5)


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [9]:
# Sort and display the five worst performing schools by % overall passing
bottom_schools_df = final_school_summary_df.sort_values("% Overall Passing", ignore_index=True)
bottom_schools_df.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [10]:
# Create a series for each grade (9-12)
ninth_grade_df = schools_df.loc[schools_df['grade'] == '9th', :]
tenth_grade_df = schools_df.loc[schools_df['grade'] == '10th', :]
eleventh_grade_df = schools_df.loc[schools_df['grade'] == '11th', :]
twelfth_grade_df = schools_df.loc[schools_df['grade'] == '12th', :]

# Group each series by school and average the math score
ninth_grade_group = ninth_grade_df.groupby('school_name')
ninth_grade_average_math = ninth_grade_group['math_score'].mean()

tenth_grade_group = tenth_grade_df.groupby('school_name')
tenth_grade_average_math = tenth_grade_group['math_score'].mean()

eleventh_grade_group = eleventh_grade_df.groupby('school_name')
eleventh_grade_average_math = eleventh_grade_group['math_score'].mean()

twelfth_grade_group = twelfth_grade_df.groupby('school_name')
twelfth_grade_average_math = twelfth_grade_group['math_score'].mean()


In [11]:
# Combine the series into a dataframe
math_scores_df = pd.DataFrame({       '9th': ninth_grade_average_math,
                                     '10th': tenth_grade_average_math,
                                     '11th': eleventh_grade_average_math,
                                     '12th': twelfth_grade_average_math
                                 })
# Rename the index column for aesthetics
math_scores_df.index.names = ['School Name']

In [12]:
math_scores_df.head(20)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade 

In [13]:
# average the math scores using school groups creating in previous section
ninth_grade_average_reading = ninth_grade_group['reading_score'].mean()

tenth_grade_average_reading = tenth_grade_group['reading_score'].mean()

eleventh_grade_average_reading = eleventh_grade_group['reading_score'].mean()

twelfth_grade_average_reading = twelfth_grade_group['reading_score'].mean()


In [14]:
# Combine the series into a dataframe
reading_scores_df = pd.DataFrame({   '9th': ninth_grade_average_reading,
                                     '10th': tenth_grade_average_reading,
                                     '11th': eleventh_grade_average_reading,
                                     '12th': twelfth_grade_average_reading
                                 })
# Rename the index column for aesthetics
reading_scores_df.index.names = ['School Name']

In [15]:
reading_scores_df.head(20)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [16]:
# Find the best bins to use for this data
school_summary_df['Per Student Budget'].describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
Name: Per Student Budget, dtype: float64

In [17]:
# Create bins in which to place values based upon the quartile in the statistical summary above
bins = [570, 590, 627, 640, 660]
        
# Create labels for these bins
group_labels = ["$570-590", "$591-627", "$628-640", "$641-660"]

# Slice the data and place it into the bins
pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_labels)

# Place the data series into a new column inside of the DataFrame
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_labels)


In [18]:
spending_group = school_summary_df.groupby('Spending Ranges (Per Student)')

# Get the average of each column within the GroupBy object
spending_ranges_scores = spending_group[['Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Overall Passing']].mean()
spending_ranges_scores.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
$570-590,83.455399,83.933814,93.460096,96.610877,90.369459
$591-627,83.516957,83.862393,93.951362,96.313180,90.344034
$628-640,78.505315,81.696400,73.076824,85.050359,62.945755
$641-660,77.023555,80.957446,66.701010,80.675217,53.717613


## Scores by School Size

In [19]:
# Create bins in which to place values
bins = [400, 999, 1999, 5000]
        
# Create labels for these bins
group_labels = ["Small (400-999)", "Medium (1000-1999)", "Large (2000-5000)"]

# Slice the data and place it into the bins
pd.cut(school_summary_df["Total Students"], bins, labels=group_labels)

# Place the data series into a new column inside of the DataFrame
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], bins, labels=group_labels)


In [20]:
size_group = school_summary_df.groupby('School Size')

# Get the average of each column within the GroupBy object
school_size_scores = size_group[['Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Overall Passing']].mean()
school_size_scores.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (400-999),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-1999),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [21]:
# merge the District back into the school_summary_df
complete_schools_df = pd.merge(school_summary_df, school_data[['school_name','type']], how='left', on=['school_name'])

# Transform the school type into categories so I can use it for binning the data (can't bin/cut on categorical data)
complete_schools_df["type"] = complete_schools_df["type"].astype('category')
complete_schools_df['type_cat'] = complete_schools_df['type'].cat.codes


In [22]:
# Create School Type bins & labels 
bins = [-0.1, 0.9, 1.1]
group_labels = ["Charter", "District"]

# Slice the data and place it into the bins
pd.cut(complete_schools_df["type_cat"], bins, labels=group_labels)

# Place the data series into a new column inside of the DataFrame
complete_schools_df["School Type"] = pd.cut(complete_schools_df["type_cat"], bins, labels=group_labels)


In [23]:
school_type_group = complete_schools_df.groupby('School Type')

# Get the average of each column within the GroupBy object
school_type_scores = school_type_group[['Average Math Score', 'Average Reading Score', '% Passing Math','% Passing Reading', '% Overall Passing']].mean()
school_type_scores.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
